In [1]:
import multiprocessing
from multiprocessing import Pool
import pandas as pd
import os
import subprocess
import glob
import json
import numpy as np
import time
from sklearn.metrics import mean_squared_error  
import math  
import warnings
from datasets import load_dataset
import jiwer
import torch
import accelerate
warnings.simplefilter(action='ignore', category=FutureWarning)
!module load openmind/ffmpeg/20160310 
from importlib import reload
import librosa
import transformers
reload(transformers)
reload(transformers.models)

reload(librosa)

<module 'librosa' from '/home/arjunp/.conda/envs/codebase3/lib/python3.8/site-packages/librosa/__init__.py'>

In [39]:
!nvidia-smi

Sat Sep 28 16:44:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  |   00000000:3B:00.0 Off |                  N/A |
| 32%   40C    P8             13W /  250W |    4051MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import sys
print(sys.executable)

/home/arjunp/.conda/envs/codebase3/bin/python


In [4]:
list_of_files = glob.glob("/om/user/arjunp/smallFineTune/data/*/*")
text = ['hello', 'how are you']
for i in list_of_files:
    df = pd.DataFrame()
    df['text'] = text
    csv_path = "/om/user/arjunp/fakeTranscripts/"+os.path.basename(i)[:-4]+".csv"
    df.to_csv(csv_path, index=False)

In [5]:
datasetdir = '/om/user/arjunp/smallFineTune'
datadir = '/om/user/arjunp/smallFineTune/data'
golddir = '/om/user/arjunp/fakeTranscripts'
transcriptlist = []
filepathlist = []

for split in os.listdir(datadir):
    split_path = os.path.join(datadir, split)
    for file_path in os.listdir(split_path):
        name = os.path.join("data", os.path.basename(split_path),os.path.basename(file_path))
        goldpath = file_path[:-4]+".csv"
        df = pd.read_csv(os.path.join(golddir,goldpath))
        transcriptlist.append(df.text.str.cat(sep=' '))
        filepathlist.append(name)

transcriptdf = pd.DataFrame(list(zip(filepathlist, transcriptlist)), columns =['file_name', 'transcription'])
transcriptdf.to_csv('/om/user/arjunp/smallFineTune/metadata.csv', index=False)

In [2]:
dataset = load_dataset("audiofolder", data_dir="/om/user/arjunp/fineTuningDataFolders/smallFineTune")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 5
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 5
    })
})


In [5]:
dataset['test'][0]

{'audio': {'path': '/weka/scratch/weka/cpl/arjunp/smallFineTune/data/test/1029_xxxx_139930.wav',
  'array': array([-0.04031372,  0.03128052,  0.040802  , ..., -0.0015564 ,
         -0.00521851, -0.00189209]),
  'sampling_rate': 16000},
 'transcription': 'hello how are you'}

In [3]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small.en")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small.en",   task="transcribe")

In [4]:
input_str = dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)
print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 hello how are you
Decoded w/ special:    <|startoftranscript|><|transcribe|><|notimestamps|>hello how are you<|endoftext|>
Decoded w/out special: hello how are you
Are equal:             True


In [6]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small.en",  task="transcribe")

In [46]:
labeldict = {"input_ids": labels}
teens = processor.tokenizer.pad(labeldict, return_tensors="pt")
teens['input_ids'].shape
len(labels)

571

In [5]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
def prepare_dataset(batch):
    audio = batch["audio"]
    #max_length=1024,padding='max_length'
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0] #log-Mel input features
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

dataset  = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"])

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [17]:
print(dataset )

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 6
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3
    })
})


In [8]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small.en")
model.generation_config.language = "english"
model.generation_config.task = "transcribe"
#model.config.max_length = 2048
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False
#model.config.max_target_positions = 2048
#model.config.max_source_positions = 2048

In [43]:
model.config

WhisperConfig {
  "_name_or_path": "openai/whisper-small.en",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50256
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50257,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50256,
  "forced_decoder_ids": null,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mask_time_prob": 0.05,
  "max_length": 448,
  "max_source_positions": 1500,
  "max_target_positions

In [89]:
from scipy.signal import resample
a = np.array(dataset['train'][0]['input_features']) 
b = resample(a, 2048, axis=1)

In [91]:
b.shape

(80, 2048)

In [9]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
          
        #audio input features to torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        
        #resampling attempt below
        '''
            for i in range(len(input_features)):
                a = np.array(input_features[i]['input_features']) 
                b = resample(a, 2048, axis=1)
                input_features[i]['input_features'] = b
        '''

        #print(np.array(input_features[0]['input_features']).shape)
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt" , padding=True )

        #tokenized label sequences and padding
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt" , padding=True)
         

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        #for i in batch["labels"]:
            #print(len(i))
        return batch


data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor,decoder_start_token_id=model.config.decoder_start_token_id)

In [10]:
import evaluate
metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


In [11]:
## For the actual training loop:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/om/user/arjunp/whisper-small-finetuned-dummy",  
    per_device_train_batch_size=6, #changing this 
    gradient_accumulation_steps=1,   
    learning_rate=1e-5,
    warmup_steps=500,
    #max_steps=5000,
    num_train_epochs = 10, #added new
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    #generation_max_length=1024, #changed to 700
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to= "none",  #changed to none from wandb
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, #changed to false
    remove_unused_columns=False #added this new arg
)


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
     
)

trainer.train()
trainer.save_model('./whisper-small-finetuned-dummy')

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/arjunp/.conda/envs/codebase3/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50256]}


In [12]:
model2 = WhisperForConditionalGeneration.from_pretrained("/om/user/arjunp/whisper-small-finetuned-dummy")

In [14]:
device = torch.device('cpu')
model.to(device)
model2.to(device)
original_model_weights = model.state_dict() 
new_model_weights = model2.state_dict() 
for layer in original_model_weights:
    if  not torch.equal(original_model_weights[layer], new_model_weights[layer]):
        print(f"Layer {layer} has changed.")